In [ ]:
import pandas as pd 
import numpy as np
import os as os
import sklearn as sklearn

import matplotlib.pyplot as plt
import seaborn as sns
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

if os.getlogin()=="JVARGH7":
    path_equity_precision_llm_folder = "C:/Cloud/OneDrive - Emory University/Papers/Global Equity in Diabetes Precision Medicine LLM"
    path_equity_precision_llm_repo =  'C:/code/external/equity_precision_llm'
if os.getlogin()=='aamnasoniwala':
    path_equity_precision_llm_folder = '/Users/aamnasoniwala/OneDrive-EmoryUniversity/Global Equity in Diabetes Precision Medicine LLM'
    path_equity_precision_llm_repo = '/Users/aamnasoniwala/code/equity_precision_llm'



In [ ]:
# Read base_prompt files from the prompts folder

base_prompt_files = ['p1v1', 'p2v1', 'p3v1']

base_prompts = []

for index, prompt_file in enumerate(base_prompt_files):
    print(index)
    print(prompt_file)
    # Read the files as a list and save it to base_prompts
    file_path = path_equity_precision_llm_repo + "/prompts/" + prompt_file + '.txt'


    with open(file_path, "r") as file:
        # Create a base_prompts list of the three prompts saved as an index 
        # base_prompts[0]: p1v1.txt
        # base_prompts[1]: p2v1.txt
        # base_prompts[2]: p3v1.txt
        base_prompts[index] = file.read()

print(base_prompts)
    



Read the PMID's abstract, title and mesh terms

In [ ]:
def prompt_generator(PMID, base_prompt_list, excel_path):
    print(PMID)

    # Step 1: Read the excel sheet
df = pd.read_excel(excel_path)

    # Step 2: Filter the excel sheet for the PMID, and create 3 variables: title, abstract, mesh
record = df[df['PMID'] == PMID].iloc[0]
title = record['Title']
abstract = record['Abstract']
mesh = record['MeSH Terms']

    # Step 3: Generate the prompt based on the PMID and source_population
prompt_template = base_prompt_list[2] 

    # For base_prompt_list[2] (i.e. p3vN.txt) 
    # Replace <INSERT TITLE> with the title (title)
    # Replace <INSERT ABSTRACT> with the abstract (abstract)
    # Replace <INSERT MESH TERMS> with the MeSH terms (mesh)

    # Use regular expressions to replace the text in the base prompt with the title, abstract, and MeSH terms
prompt_new = re.sub(r"<INSERT TITLE>", title, prompt_template)
prompt_new = re.sub(r"<INSERT ABSTRACT>", abstract, prompt_new)
prompt_new = re.sub(r"<INSERT MESH TERMS>", mesh, prompt_new)
    # Step 4: Return the prompt
return prompt_new


pmid_list = [22744164, 15561964, 28770629, 33764184, 36155119, 35477306, 23153210, 34794279, 12116231, 2109332, 6453040, 31807629, 28068033, 28668296, 34872002, 32663752, 12475769, 22113416, 12638530, 36649380]  # Replace with PMIDs
excel_path = "/aamnasoniwala/Library/CloudStorage/OneDrive-Emory/Global Equity in Diabetes Precision Medicine LLM/llm training/Methods.xlsx"
prompt_pmid = prompt_generator(PMID, base_prompts, excel_path)

# Generate loop
for PMID in pmid_list:
    # Generate the customized prompt
    prompt_pmid = prompt_generator(PMID, base_prompts, excel_path)

    # Create a directory for each PMID in the working folder
    pmid_folder = os.path.join(path_equity_precision_llm_folder, "working", str(PMID))
    os.makedirs(pmid_folder, exist_ok=True)


    # Define file paths for each prompt
    file_path_p1 = os.path.join(pmid_folder, f"{base_prompt_files[0]}_{PMID}.txt")
    file_path_p2 = os.path.join(pmid_folder, f"{base_prompt_files[1]}_{PMID}.txt")
    file_path_p3 = os.path.join(pmid_folder, f"{base_prompt_files[2]}_{PMID}.txt")


# Because we are keeping p1vN, we save the original file
with open(file_path_p1, "w") as file:
        file.write(base_prompts[0])

# Because we are keeping p2vN, we save the original file
with open(file_path_p2, "w") as file:
        file.write(base_prompts[1])

# Because we are modifying p3vN, we need to save the modified prompt to a new file
with open(file_path_p3, "w") as file:
        file.write(prompt_pmid)

print(f"Prompts generated and saved for PMID {PMID}")
